# 深層学習ノートブック-6 誤差逆伝播(Backpropagation)
誤差逆伝播をスクラッチで実装してみる。  
まず大前提として、誤差逆伝播は損失が少なくなるようにパラメータを更新するためのものであるから、  
$\frac{\partial{L}}{\partial{W^{[l]}}}$ と $\frac{\partial{L}}{\partial{B^{[l]}}}$ をいかに求めるかが実装する上で重要なところ。  

* $l$番目の層における損失の誤差 
$$\delta^{[l]}=\frac{\partial{L}}{\partial{Z^{[l]}}}$$  
$$=\frac{\partial{L}}{\partial{A^{[l]}}} \bigodot \frac{\partial{A}}{\partial{Z^{[l]}}}$$
$$=(\delta^{[l+1]}\textbf{W}^{[l+1]}) \bigodot \sigma'(\textbf{Z}^{[l]})$$  

* パラメタの勾配
$$\frac{\partial{L}}{\partial{W^{[l]}}}=\delta^{[l]T}\textbf{A}^{[l-1]}$$  
$$\frac{\partial{L}}{\partial{b^{[l]}}}=\sum_i\delta_i^{[l]}$$  

* パラメタの更新
$$\textbf{W}^{[l]}=\textbf{W}^{[l]}-\alpha\frac{\partial{L}}{\partial{W}^{[l]}}$$  
$$\textbf{B}^{[l]}=\textbf{B}^{[l]}-\alpha\frac{\partial{L}}{\partial{B}^{[l]}}$$  


※ReLUの導関数
\begin{equation}
\sigma^{'}(z)= \left \{
\begin{array}{l}
1　(if z > 0) \\
0　(otherwise)
\end{array}
\right.
\end{equation}

In [80]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

## ■ backward（逆伝播）の実装

In [82]:
# 線形変換部分(パラメタ)の勾配の算出関数。Aはl-1層目の値で他はl層目の値。
def linear_backward(A, W, b, Z):
     # pytorch.tensorの.grad属性と区別するために.grad_として属性を定義している
     W.grad_ = Z.grad_.T @ A
     # bの値が一つ変わると、全データ分Lへ影響を及ぼすので、行方向(データ数の方向)にZ.grad_を足す。
     b.grad_ = torch.sum(Z.grad_, dim=0)
     # l-1層目のAはl層目の誤差（=Z.grad_）とl層目のWから計算できる。
     A.grad_ = Z.grad_ @ W


# ReLU関数
def relu_backward(Z, A):
     # 上式の"l番目の層における損失の誤差"における2行目と3行目を組み合わせてdL/dZを算出している
     # A.grad_はdL/dA, (Z > 0).float()はReLu関数の偏微分に相当する。
     Z.grad_ = A.grad_ * ( Z > 0 ).float()

### 補足1
tensor * (tensor > 0)の計算について

In [83]:
a = torch.randn((2,3)) 
b = torch.randn((2,3)) 
print(a)
print(b)
print(b > 0)

tensor([[ 0.1594,  1.7124, -1.4743],
        [ 0.4990,  0.4816,  1.3858]])
tensor([[ 0.5913, -1.5723, -0.5917],
        [-0.5025, -0.0819, -0.7331]])
tensor([[ True, False, False],
        [False, False, False]])


In [84]:
a * (b > 0)

tensor([[0.1594, 0.0000, -0.0000],
        [0.0000, 0.0000, 0.0000]])

このようにb > 0がTrueの要素は1、Falseの要素は0として計算される。

上記のように損失、パラメタの勾配をコードで表わすことが出来た。  
ではこれらの計算を実際に実行するにはどうすればよいか？  
当然、具体的な計算を行うためには上記の関数の入力であるA,W,b,Zを求める必要がある。  
すなわち順伝播(forward)の計算を行い、損失を計算してから逆伝播(backward)の計算を行うことになる。  

まず損失を求める関数を実装する。

In [85]:
# softmax関数と交差エントロピーの計算を同じ関数で実装する（pytorchでもこうなっている）
def softmax_cross_entropy(X, y_true):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    y_true: tensor。One-Hot Encoding済みの正解ラベル。
    '''
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10
    softmax_out = e_x / denominator
    loss = - (y_true * torch.log(softmax_out + 1e-10)).sum() / y_true.shape[0]

    return loss, softmax_out


In [86]:
# tensorの線形結合を返す関数
def linear_comb(X, W, B):
    '''
    X, W, B: torch.tensor
    '''
    return X @ W.T + B

In [87]:
# ReLUの実装
def ReLU(Z):
    '''
    Z: torch.tensor
    '''
    # torch.whereによって要素ごとに条件が真・偽のときで別の値を返せる
    # 下記では0より大きい要素はzの値そのままで、0以下は0.になる。
    return torch.where(Z > 0 , Z, 0.)

順伝播→逆伝播の計算を行う関数を定義。  
中間層の勾配もスクラッチの結果と等しいか確認したいので、  
.retain_grad()を次の計算の前に呼び出している。  

In [89]:
# # 隠れ層１層の場合のMLP
# def forward_and_backward(X, W_list, B_list, y):
#     '''
#     X: features
#     W: List of weights(each edge)
#     B: List of Bias(each Layer)
#     '''

#     #########################
#     #### 順伝播（forward）####
#     #########################

#     # 入力層→隠れ層
#     Z1 = linear_comb(X, W_list[0], B_list[0])
#     Z1.retain_grad()

#     # 活性化関数適用
#     A1 = ReLU(Z1)
#     A1.retain_grad()

#     # 隠れ層→出力層
#     Z2 = linear_comb(A1, W_list[1], B_list[1])
#     Z2.retain_grad()

#     # 最終出力。損失とソフトマックス関数の出力が返される
#     loss, A2 = softmax_cross_entropy(Z2, y)


#     #########################
#     #### 逆伝播（backward）####
#     #########################

#     # 最終出力→出力層の出力のbackward。ここで各層の出力の勾配を求める。
#     # ここでは出力層の活性化関数は恒等関数とし、
#     # その出力にsoftmax_cross_entropyを適用した結果をモデルの最終出力として考えている。
#     Z2.grad_ = (A2 - y) / X.shape[0]
#     # 出力層→隠れ層のbackward
#     linear_backward(A1, W_list[1], B_list[1], Z2)
#     # 隠れ層の出力側のbackward
#     relu_backward(Z1, A1)
#     # 隠れ層の入力側のbackward
#     linear_backward(X, W_list[0], B_list[0], Z1)
    
#     return loss, Z1, A1, Z2, A2
    

In [90]:
# 隠れ層１層の場合のMLP（スクラッチ）
def forward_and_backward(X, W_1, W_2, B_1, B_2, y):
    '''
    X: features
    W: List of weights(each edge)
    B: List of Bias(each Layer)
    '''

    #########################
    #### 順伝播（forward）####
    #########################

    # 入力層→隠れ層
    Z1 = linear_comb(X, W_1, B_1)
    Z1.retain_grad()

    # 活性化関数適用
    A1 = ReLU(Z1)
    A1.retain_grad()

    # 隠れ層→出力層
    Z2 = linear_comb(A1, W_2, B_2)
    Z2.retain_grad()

    # 最終出力。損失とソフトマックス関数の出力が返される
    loss, A2 = softmax_cross_entropy(Z2, y)


    #########################
    #### 逆伝播（backward）####
    #########################

    # 最終出力→出力層の出力のbackward。ここで各層の出力の勾配を求める。
    # ここでは出力層の活性化関数は恒等関数とし、
    # その出力にsoftmax_cross_entropyを適用した結果をモデルの最終出力として考えている。
    Z2.grad_ = (A2 - y) / X.shape[0]
    # 出力層→隠れ層のbackward
    linear_backward(A1, W_2, B_2, Z2)
    # 隠れ層の出力側のbackward
    relu_backward(Z1, A1)
    # 隠れ層の入力側のbackward
    linear_backward(X, W_1, B_1, Z1)
    
    return loss, Z1, A1, Z2, A2
    

### 補足：Z2.grad_をデータ数X.shape[0]で割る理由  
交差エントロピーで算出した全体の損失Lはデータ数で割った平均の形なので、右辺もデータ数で割る必要あり。  
各データに対応する損失をデータ数で割ることで、最終的な全体の損失Lへ与える影響をならしているイメージ。  
誤差逆伝播の講義資料のdL/dZ[2]の式を参照。両辺に1/mが実際はかかっているということ。  

## ■ Autogradの結果と一致することを確認
MNISTデータセットを使って、順伝播後に逆伝播を計算する。  
前処理部分は前回までのコードの使いまわし。

In [95]:
# 変数定義
learning_rate = 0.03
loss_log = []  #学習時の損失記録用のリスト

# データロード
dataset = datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32)
target = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of y_train: {target.shape}')
print(target)

# 目的変数のエンコーディング
y_true = F.one_hot(target, num_classes=10)
print(f'shape of y_true: {y_true.shape}')

# 学習データと検証データを8:2に分ける
X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

print(f'shape of train data: X_train:{X_train.shape}, y_train:{y_train.shape}')
print(f'shape of validation data: X_val:{X_val.shape}, y_val:{y_val.shape}')

# 学習データ・検証データの標準化。検証データの標準化には学習データの平均、標準偏差を使用することに注意。  
X_mean = X_train.mean()
X_std = X_train.std()
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std

# データ数、特徴量数、隠れ層のノード数、最終的な出力数（ここではクラス数）を定義
m, nf = X_train.shape
nh = 30
n_out = 10

# 入力層→隠れ層の重みW、バイアス項bの初期化
W_1 = torch.randn(size=(nh, nf) ,requires_grad=True) #出力×入力
B_1 = torch.zeros(size=(1, nh), requires_grad=True) # 1 x 出力

# 隠れ層→出力層の重みW、バイアス項bの初期化
W_2 = torch.randn(size=(n_out, nh) ,requires_grad=True) #出力×入力
B_2 = torch.zeros(size=(1, n_out), requires_grad=True) # 1 x 出力
# W_list = [W_1, W_2]
# B_list = [B_1, B_2]


shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.])
shape of y_train: torch.Size([1797])
tensor([0, 1, 2,  ..., 8, 9, 8])
shape of y_true: torch.Size([1797, 10])
shape of train data: X_train:torch.Size([1437, 64]), y_train:torch.Size([1437, 10])
shape of validation data: X_val:torch.Size([360, 64]), y_val:torch.Size([360, 10])


In [96]:
# スクラッチで実装した順伝播→逆伝播の関数を実行。
# これでスクラッチバージョンの勾配(.grad_)が得られる。
loss, Z1, A1, Z2, A2 = forward_and_backward(X_train, W_1, W_2, B_1, B_2, y_train)

# autogradバージョンの逆伝播(.grad)を得たいのならば、普通に最終的なlossの結果に.backward()するでOK
loss.backward()

In [97]:
# autogradのdL/dZ2の結果
Z2.grad

tensor([[ 3.4096e-10,  7.6112e-17,  1.4405e-20,  ...,  6.9581e-04,
          7.5372e-31,  7.2105e-34],
        [-3.4070e-16,  7.9003e-38,  0.0000e+00,  ...,  3.4070e-16,
          0.0000e+00,  0.0000e+00],
        [-1.3026e-14,  9.1311e-36,  0.0000e+00,  ...,  1.3026e-14,
          4.2179e-43,  0.0000e+00],
        ...,
        [ 2.0859e-20,  4.4915e-34, -1.5268e-13,  ...,  2.0861e-33,
          1.1953e-42,  2.8026e-45],
        [ 2.4493e-18,  2.7161e-06,  5.0713e-30,  ..., -6.9585e-04,
          1.5190e-32,  1.2584e-23],
        [ 2.1766e-26, -6.9558e-04,  7.1603e-21,  ...,  5.7043e-24,
          1.7906e-29,  8.6660e-29]])

In [98]:
# スクラッチのdL/dZ2の結果
Z2.grad_

tensor([[ 3.4096e-10,  7.6112e-17,  1.4405e-20,  ...,  6.9581e-04,
          7.5372e-31,  7.2105e-34],
        [-6.9589e-04,  1.6137e-25,  1.0536e-37,  ...,  6.9589e-04,
          1.6675e-43,  1.2831e-33],
        [-6.9589e-04,  4.8779e-25,  2.1961e-35,  ...,  6.9589e-04,
          2.2568e-32,  1.1918e-39],
        ...,
        [ 9.5073e-11,  2.0472e-24, -6.9589e-04,  ...,  9.5082e-24,
          5.4504e-33,  1.0563e-35],
        [ 2.4493e-18,  2.7161e-06,  5.0713e-30,  ..., -6.9585e-04,
          1.5190e-32,  1.2584e-23],
        [ 2.1766e-26, -6.9558e-04,  7.1603e-21,  ...,  5.7043e-24,
          1.7906e-29,  8.6660e-29]], grad_fn=<DivBackward0>)

In [99]:
# torch.allcloseで大体同じか否かを判定できる。
torch.allclose(Z2.grad, Z2.grad_)

False

In [100]:
Z2

tensor([[ 15.1223,  -0.1928,  -8.7652,  ...,  29.6511, -32.4388, -39.3908],
        [  5.4322,   6.9875, -21.0698,  ...,  56.8033, -34.4301, -11.6624],
        [  7.9314,   6.9492, -16.8747,  ...,  55.6588,  -9.9397, -26.6963],
        ...,
        [ 19.7737, -11.6955,  -9.6862,  ..., -10.1598, -31.4395, -37.6856],
        [ -2.7247,  25.0097, -29.6279,  ...,  20.9228, -35.4386, -14.9036],
        [-16.7597,  27.3490,  -4.0560,  ..., -11.1911, -23.8626, -22.2858]],
       grad_fn=<AddBackward0>)

Z2の勾配を計算すると、  
スクラッチとautogradでデータによって同じだったり、異なったりしていることが分かる。  

深層学習の問題点の一つとして、Xに始まる入力に対して各層で線形結合を繰り返すと    
出力結果が元のXの分布からどんどん大きくなっていき、 大きくなり過ぎた出力を最終的にsoftmax関数へ入れることで、  
eの累乗の計算が大きくなり過ぎて勾配計算に誤差が出るという点がある。  
上記におけるsoftmax関数の入力はZ2であり、Z2を見ると指数としては大きい値になっていることが分かる。

上記ではsoftmax関数への入力が非常に大きい場合の取り扱い方が、  
スクラッチ実装とpytorchの.backward()の実装で恐らく違いがあるため、上記のようにところどころ計算が合わなくなったのだろう。  

## ■ 一致しないことに対する対処
出力が大きくなり過ぎないための工夫として、  
重みの初期化の際にrandn（標準正規分布からのサンプリング）ではなく、  
平均0, 分散$\frac{2}{n}$(nは特徴量数) の正規分布から取り出すというものが知られている。（下記）  
なぜこれでうまくいくのかは後述。  

In [102]:
# データ数、特徴量数、隠れ層のノード数、最終的な出力数（ここではクラス数）を定義
m, nf = X_train.shape
nh = 30
n_out = 10

# 入力層→隠れ層の重みW、バイアス項bの初期化
# W_1 = torch.randn(size=(nh, nf) ,requires_grad=True) #出力×入力

# randnは分散が1なので、2/nのルートをかければよい。
W_1 = torch.randn((nh, nf)) * torch.sqrt(torch.tensor(2./nf))
W_1.requires_grad = True # 一つ上のrandnでTrueにしても計算後のtensorに引き継がれないので別途設定する必要あり。
B_1 = torch.zeros(size=(1, nh), requires_grad=True) # 1 x 出力

# 隠れ層→出力層の重みW、バイアス項bの初期化
#W_2 = torch.randn(size=(n_out, nh) ,requires_grad=True) #出力×入力
W_2 = torch.randn((n_out, nh)) * torch.sqrt(torch.tensor(2./nf))
W_2.requires_grad = True 

B_2 = torch.zeros(size=(1, n_out), requires_grad=True) # 1 x 出力
# W_list = [W_1, W_2]
# B_list = [B_1, B_2]


In [103]:
# 上記の工夫を踏まえて再計算
loss, Z1, A1, Z2, A2 = forward_and_backward(X_train, W_1, W_2, B_1, B_2, y_train)
loss.backward()

In [104]:
# autogradの結果
W_1.grad

tensor([[-0.0297, -0.0325, -0.0345,  ...,  0.0139, -0.0218, -0.0273],
        [ 0.0149,  0.0135, -0.0075,  ...,  0.0006,  0.0145,  0.0135],
        [-0.0163, -0.0158, -0.0057,  ...,  0.0050, -0.0110, -0.0192],
        ...,
        [-0.0068, -0.0050,  0.0362,  ...,  0.0109, -0.0066, -0.0065],
        [-0.0153, -0.0136,  0.0118,  ...,  0.0141, -0.0152, -0.0173],
        [-0.0013, -0.0012, -0.0020,  ..., -0.0097, -0.0072, -0.0025]])

In [105]:
# スクラッチ実装の結果
W_1.grad_

tensor([[-0.0297, -0.0325, -0.0345,  ...,  0.0139, -0.0218, -0.0273],
        [ 0.0149,  0.0135, -0.0075,  ...,  0.0006,  0.0145,  0.0135],
        [-0.0163, -0.0158, -0.0057,  ...,  0.0050, -0.0110, -0.0192],
        ...,
        [-0.0068, -0.0050,  0.0362,  ...,  0.0109, -0.0066, -0.0065],
        [-0.0153, -0.0136,  0.0118,  ...,  0.0141, -0.0152, -0.0173],
        [-0.0013, -0.0012, -0.0020,  ..., -0.0097, -0.0072, -0.0025]],
       grad_fn=<MmBackward0>)

In [106]:
# autogradと大体等しいことを確認
print(torch.allclose(W_1.grad_, W_1.grad))
print(torch.allclose(B_1.grad_, B_1.grad))
print(torch.allclose(W_2.grad_, W_2.grad))
print(torch.allclose(B_2.grad_, B_2.grad))
print(torch.allclose(Z1.grad_, Z1.grad))
print(torch.allclose(Z1.grad_, Z1.grad))


True
True
True
True
True
True


工夫前と異なり、autogradと結果が等しくなった。

In [107]:
Z2

tensor([[ 1.0273, -1.0354, -0.5734,  ..., -0.1903, -1.3035, -0.9548],
        [ 0.2881, -0.1615, -0.9772,  ..., -0.0853, -0.3612, -0.9244],
        [ 0.7111, -0.9237, -1.0729,  ...,  0.7416, -1.7103, -1.1776],
        ...,
        [ 0.7833, -0.5741,  0.4119,  ...,  0.0655, -1.5887,  0.3286],
        [ 0.2352, -0.1118, -0.3671,  ...,  0.4156, -1.2270, -0.6012],
        [-1.7943, -0.2663,  1.3029,  ..., -0.1775, -0.6772, -0.8382]],
       grad_fn=<AddBackward0>)

工夫の結果、softmaxへの入力Z2がそれほど大きくならなくなったことが分かる。